## ДАТАСЕТ

В данном блоке мы будем работать с данными о компании, организующей ***перевозки*** грузов.

Интересующие данные хранятся в таблицах:

- city (справочник городов)
    * city_id	integer	уникальный идентификатор города, первичный ключ
    * city_name	text	название города
    * state	text	штат, к которому относится город
    * population	integer	население города
    * area	numeric	площадь города
    
- customer (справочник клиентов)
    * cust_id	integer	уникальный идентификатор клиента, первичный ключ
    * cust_name	text	название клиента
    * annual_revenue	numeric	ежегодная выручка
    * cust_type	text	тип пользователя
    * address	text	адрес
    * zip	integer	почтовый индекс
    * phone	text	телефон
    * city_id	integer	идентификатор города, внешний ключ к таблице city

- driver (справочник водителей)
    * driver_id	integer	уникальный идентификатор водителя, первичный ключ
    * first_name	text	имя водителя
    * last_name	text	фамилия водителя
    * address	text	адрес водителя
    * zip_code	integer	почтовый индекс водителя
    * phone	text	телефон водителя
    * city_id	integer	идентификатор города водителя, внешний ключ к таблице city

- shipment (таблица с данными непосредственно о доставках)
    * ship_id	integer	уникальный идентификатор доставки, первичный ключ
    * cust_id	integer	идентификатор клиента, которому отправлена доставка, внешний ключ к таблице customer
    * weight	numeric	вес посылки
    * truck_id	integer	идентификатор грузовика, на котором отправлена доставка, внешний ключ к таблице truck
    * driver_id	integer	идентификатор водителя, который осуществлял доставку, внешний ключ к таблице driver
    * city_id	integer	идентификатор города в который совершена доставка, внешний ключ к таблице city
    * ship_date	date	дата доставки

- truck (информация о грузовиках, на которых осуществляются перевозки)
    * truck_id	integer	Уникальный идентификатор грузовика, первичный ключ
    * make	text	Производитель грузовика
    * model_year	integer	Дата выпуска грузовика

``` sql

-- Название города с максимальным весом единичной доставки

select
c.city_name,
s.weight
from
sql.city c
inner join sql.shipment s on c.city_id = s.city_id
order by weight desc
limit 1

-- Сколько различных производителей грузовиков перечислено в таблице truck

select
count(distinct t.make) make_cnt
from
sql.truck t

-- Как зовут водителя (first_name), который совершил наибольшее количество доставок одному клиенту?

select
d.first_name,
s.cust_id,
count(*) max_count_deliver
from
sql.driver d
inner join sql.shipment s on d.driver_id = s.driver_id
group by d.first_name, s.cust_id
order by count(*) desc
limit 1

-- Укажите даты первой и последней по времени доставок в таблице shipment

select
min(s.ship_date) first_date,
max(s.ship_date) last_date
from
sql.shipment s

-- Укажите имя клиента, получившего наибольшее количество доставок за 2017 год

select
c.cust_name,
count(c.cust_id)
from
sql.customer c
inner join sql.shipment s on c.cust_id = s.cust_id
where to_char(s.ship_date, 'YYYY') = '2017'
group by c.cust_name
order by count(c.cust_id) desc
limit 1
```

## ПРИНЦИП И УСЛОВИЯ РАБОТЫ UNION

Примечание: Важно, чтобы при объединении таблиц в них был одинаковый тип данных и одинаковое количество колонок!

UNION никогда не уменьшает исходную таблицу: он либо оставляет ее неизменной, либо увеличивает ее длину.

Допустим, мы хотим собрать из справочников по книгам и фильмам один, так чтобы в нём содержались названия произведений, а также их описание — книга или фильм.

Для этого напишем запрос:

``` sql

select book_name object_name, 'книга' object_describtion -- <--- Головной запрос самый важный, он задает: алиасы, названия колонок, количество колонок, тип данных
from
public.books
union all
select movie_title, 'фильм'
from
sql.kinopoisk

-- Выборка в такой объедиенной таблице следующая

(select book_name object_name, 'книга' object_describtion -- < Головной запрос самый важный, он задает: алиасы, названия колонок, количество колонок, тип данных
from
public.books
order by 1) -- < Это отсортирует первую таблицу, при этом, заключается в круглые скобки
union all
(select movie_title, 'фильм'
from
sql.kinopoisk
order by 1) -- < Это отсортирует вторую таблицу, при этом, заключается в круглые скобки
order by 1 -- < Запись в конце отсортирует всю объединенную таблицу!
```

## ВИДЫ UNION

Оператор присоединения существует в двух вариантах:

* UNION выводит только уникальные записи;
* union all присоединяет все строки последующих таблиц к предыдущим, без ограничений по уникальности.

Примечание: UNION оставляет только уникальные значения, а потому требует дополнительных вычислительных мощностей и памяти (в данном случае можно провести аналогию с DISTINCT). Поэтому если вы уверены в отсутствии дубликатов в данных или они вам не важны, предпочтительнее использовать union all.


##СИНТАКСИС

Запрос строится таким образом:

``` sql
select         n columns
from 
         table_1
union all
select 
         n columns
from 
         table_2
...
union all
select 
         n columns
from 
         table_n
```

Результатом выполнения такого запроса будут строки table_1, table_2, ..., table_n, соединённые одни под другими и выведенные в единой выдаче.

Примечание: Названия итоговых колонок в выводе будут такие же, как в первом блоке SELECT, даже если они отличаются в других блоках подзапросов.

Обратимся к нашему датасету о транспортной компании и посмотрим, как сформировать справочник с ID всех таблиц и указанием объекта, к которому он относится.

``` sql

select
c.city_id object_name, 'id города' object_type
from 
sql.city c

union all

select
d.driver_id other_name, 'id водителя' other_type
from
sql.driver d

union all

select
s.ship_id, 'id доставки'
from
sql.shipment s

union all

select
c.cust_id, 'id клиента'
from
sql.customer c

union all

select
t.truck_id, 'id грузовика'
from
sql.truck t
order by 1

```

Примечание: Несмотря на исходные названия колонок other_name и other_type во втором подзапросе, в выводе мы получим названия, которые дали в первом блоке: object_name и object_type.

Другая особенность — в применении сортировки ORDER BY: она всегда будет относиться к итоговому результату всего запроса с UNION ALL.

``` sql

-- Запрос, который создает уникальный алфавитный справочник всех городов, штатов, имён водителей и производителей грузовиков. 
-- Результатом запроса должны быть два столбца: название и тип объекта (city, state, driver, truck). 
-- Отсортируйте список по названию объекта, а затем — по типу.

select
c.city_name object_name, 'city' object_type
from 
sql.city c

union

select
c.state, 'state'
from
sql.city c

union

select
d.first_name, 'driver'
from
sql.driver d

union

select
t.make, 'truck'
from
sql.truck t
order by object_name, object_type

-- Запрос, который соберёт имена всех упомянутых городов и штатов из таблицы city. Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

select
c.city_name object_name
from 
sql.city c

union all

select
c.state
from
sql.city c
order by object_name

-- Выполнив предыдущий запрос, мы получили города с одинаковыми названиями, но находящиеся в разных штатах, а также большое количество дублирующихся названий штатов. 
-- Перепишите предыдущий запрос так, чтобы остались только уникальные названия городов и штатов. 
-- Результатом запроса должен быть один столбец object_name, отсортированный в алфавитном порядке.

select
c.city_name object_name
from 
sql.city c

union

select
c.state
from
sql.city c
order by object_name
```

Как уже известно, чтобы вывести в одном столбце объединенные данные они должны быть одного типа данных. Однако, если мы все же хотим выполнить поставленную задачу, нам необходимо привести оба столбца к одному типу данных.

Не каждый текст, например, может быть приведён к числу, зато каждое число может быть представлено в текстовом формате!

Для типизации в Postgres составляется запрос по модели: 

column_name::column_type

Таким образом, чтобы перевести city_id в текст, нам потребуется написать city_id::text.

``` sql

-- Вывести id и название в одном столбце c приведением столбца id к типу text

select
c.city_id::text
from
sql.city c

union all

select
cc.city_name
from
sql.city cc

```

Примечание: Любой тип данных может быть приведён к текстовому формату — эту возможность целесообразно использовать для соединения разнородных сущностей. Главное — помнить, что сортировка текста отличается от сортировки чисел и дат.

``` sql

-- Запрос, который объединит в себе все почтовые индексы водителей и их телефоны в единый столбец-справочник contact. 
-- Также добавьте столбец с именем водителя first_name и столбец contact_type с типом контакта (phone или zip в зависимости от типа). 
-- Отсортируйте список по столбцу с контактными данными в порядке возрастания, а затем — по имени водителя.

select
d.phone contact, d.first_name, 'phone' contact_type
from
sql.driver d

union all

select
dd.zip_code::text, dd.first_name, 'zip'
from
sql.driver dd
order by contact, first_name

```

## ВОЗМОЖНОСТИ UNION

Помимо соединения разнородных сущностей в единый справочник, UNION ALL часто используется для подведения промежуточных итогов и выведения результатов агрегатных функций.

Кроме агрегатных функций, в запросах с UNION могут использоваться функции группировки и выборки.

``` sql

-- Использование UNION с агрегатными функциями, в данном случае SUM (вывод обобщённых данных о населении по всем городам, с детализацией до конкретного города)

select
c.city_name, c.population
from
sql.city c

union all

select
'TOTAL', sum(cc.population)
from
sql.city cc
order by 2 desc

-- Запрос, который выводит общее число доставок total_shipments, а также количество доставок в каждый день. 
-- Необходимые столбцы: date_period, cnt_shipment. Не забывайте о единой типизации. 
-- Упорядочите по убыванию столбца date_period.

select
s.ship_date::text date_period, count(*) cnt_shipment
from
sql.shipment s
group by s.ship_date

union all

select
'total_shipments', count(*)
from
sql.shipment ss
order by date_period desc

```

## UNION И ДОПОЛНИТЕЛЬНЫЕ УСЛОВИЯ

UNION также может быть использован для разделения существующей выборки по критерию «выполнение определённого условия».

``` sql

-- С помощью UNION можно отобразить, у кого из водителей заполнен столбец с номером телефона

select 
d.first_name, d.last_name, 'телефон заполнен' phone_info
from
sql.driver d
where d.phone is not null -- условие для таблицы сверху

union

select 
d.first_name, d.last_name, 'телефон не заполнен' phone_info
from
sql.driver d
where d.phone is null -- условие для таблицы снизу


-- Запрос, который выведет все города и штаты, в которых они расположены, а также информацию о том, была ли осуществлена доставка в этот город:
-- если в город была осуществлена доставка, то выводим 'доставка осуществлялась';
-- если нет — выводим 'доставка не осуществлялась'.
-- Столбцы к выводу: city_name, state, shipping_status. 
-- Отсортируйте в алфавитном порядке по городу, а затем — по штату.

select
c.city_name city_name, c.state state, 'доставка осуществлялась' shipping_status
from
sql.city c
where c.city_id in (select s.city_id from sql.shipment s)
union

select
c.city_name, c.state, 'доставка не осуществлялась'
from
sql.city c
where c.city_id not in (select s.city_id from sql.shipment s)
order by city_name, state

-- Другой вариант той же задачи

select
c.city_name city_name, c.state state, 'доставка осуществлялась' shipping_status
from 
sql.city c
left join sql.shipment s on c.city_id = s.city_id
where s.city_id is not null

union

select 
c.city_name city_name, c.state state, 'доставка не осуществлялась' shipping_status
from 
sql.city c
left join sql.shipment s on c.city_id = s.city_id
where s.city_id is null
order by city_name, state

-- Запрос, который выводит два столбца: city_name и shippings_fake. Выведите города, куда совершались доставки. Пусть первый столбец содержит название города, а второй формируется так:
-- если в городе было более десяти доставок, вывести количество доставок в этот город как есть;
-- иначе — вывести количество доставок, увеличенное на пять.
-- Отсортируйте по убыванию получившегося «нечестного» количества доставок, а затем — по имени в алфавитном порядке.

select c.city_name city_name, count(*) shippings_fake
from
sql.city c
join sql.shipment s on c.city_id = s.city_id
group by c.city_id
having count(*) > 10

union

select c.city_name, count(c.city_name) + 5
from
sql.city c
join sql.shipment s on c.city_id = s.city_id
group by c.city_id
having count(*) <= 10
order by shippings_fake desc, city_name
```

## UNION и ручная генерация

UNION можно использовать для создания справочников прямо в коде запроса. К примеру, если мы хотим вручную ввести какие-то значения и произвести с ними некоторые манипуляции или дополнить существующую выдачу своими значениями.

``` sql

-- Пример выполнения ручной генерации с использованием UNION

select
'a' letter, '1' original_position

union
select
'b', '2'

union
select
'c', '3'

-- Запрос, который выберет наибольшее из значений:

-- 1000000;
-- 541;
-- -500;
-- 100.
-- Столбец с результатом назовите result

select greatest(1000000, 541, -500, 100) as result

-- Другой вариант этого же запроса

select 
1000000 result

union all
select 
541

union all
select
-500

union all
select
100
order by 1 desc
limit 1

-- Помним, что сортировка для числовых и строковых типов данных отличается. 
-- Построив запрос по аналогии с примером, приведите значения к текстовому типу данных, сравните и выберите наибольшее из них:

-- 1000000;
-- 541;
-- -500;
-- 100.
-- Столбец с ответом назовите mycol

select 
'1000000' mycol

union all
select 
'541'

union all
select
'-500'

union all
select
'100'
order by 1 desc
limit 1

Примечание: Cортировка числовых значений происходит на основе величины числа, а сортировка строковых значений сравнивается посимвольно по алфавитному порядку на основе UTF-8, ASCII.

-- Построив запрос по аналогии с примером, найдите самое большое значение из перечисленных операторов:

-- + ;
-- - ;
-- = ;
-- / .
-- Столбец с ответом назовите result

select
'+' result

union all
select
'-'

union all
select
'='

union all
select
'/'
order by 1 desc
limit 1
```

## ИСКЛЮЧАЕМ ПОВТОРЕНИЕ ДАННЫХ

Предположим, нам нужно узнать, в какие города осуществлялась доставка, **за исключением** тех, в которых проживают водители.

``` sql

select
c.city_name
from
sql.shipment s
join sql.city c on s.city_id = c.city_id

except

select
cc.city_name
from
sql.driver d
join sql.city cc on d.city_id = cc.city_id
order by 1

```

Все водители проживают в городе Memphis, и мы видим, что он не выводится в результате запроса.

Для решения этой задачи мы использовали оператор EXCEPT.

Except работает так, что первая часть запроса полностью отображается в выборке, а вторая часть, которая идет после слова except (т.е. исключить, вырезать), убирает или исключает вторую часть запроса. 

В итоге мы получаем данные за исключением той информации, которую мы указали после слова except.

Синтаксические правила для оператора EXCEPT такие же, как и для UNION:

одинаковый тип данных;
одинаковое количество столбцов;
одинаковый порядок столбцов согласно типу данных.
Синтаксис выглядит следующим образом:

``` sql
SELECT 
         n columns
FROM 
         table_1
EXCEPT
SELECT 
         n columns
FROM 
         table_2

```

Мы уже знаем, как решить такую задачу с использованием LEFT JOIN. Вариант с EXCEPT будет полезен в тех случаях, когда у вас много столбцов и вам не хочется прописывать их равенство в условии для JOIN.

``` sql

-- Выведите список zip-кодов, которые есть в таблице sql.driver, но отсутствуют в таблице sql.customer. 
-- Отсортируйте по возрастанию, столбец к выводу — zip. 
-- В поле ниже введите запрос, с помощью которого вы решили эту задачу.

select
d.zip_code zip
from
sql.driver d

except

select
c.zip
from
sql.customer c
order by 1 desc

```

## ОБЩИЕ ДАННЫЕ

А что если нам надо вывести общие записи — те, что существуют в нескольких таблицах. Это будет классическое пересечение, где мы берем данные присутствующие в обеих таблицах. Данные манипуляции мы производим с помощью оператора INTERSECT.

``` sql

-- Предположим, нам надо вывести совпадающие по названию города и штаты

select
c.city_name city_name
from
sql.city c

intersect

select
cc.state
from
sql.city cc
order by 1

```

Примечание: При выборке необходимо иметь одинаковое количество столбцов и их типы. 

INTERSECT оставляет из результатов первого запроса все строки, которые совпали с результатом выполнения второго запроса.

Синтаксис запроса с оператором INTERSECT выглядит следующим образом:

``` sql
SELECT 
         n columns
FROM 
         table_1
INTERSECT
SELECT 
         n columns
FROM 
         table_2

```

Примечание: Как EXCEPT, так и INTERSECT убирают дубликаты, если они имеются.

``` sql

-- Запрос, который выведет список id городов, в которых есть и клиенты, и доставки, и водители.

select
c.city_id
from
sql.customer c

intersect

select
s.city_id
from
sql.shipment s

intersect

select
d.city_id
from
sql.driver d

-- Выведите zip-код, который есть как в таблице с клиентами, так и в таблице с водителями.

select
c.zip
from
sql.customer c

intersect

select
d.zip_code
from
sql.driver d

```

## ИТОГ

Общая структура запроса будет выглядеть следующим образом:

``` sql

SELECT          N columns
FROM          table_1
UNION / UNION ALL / EXCEPT / INTERSECT 
SELECT          N columns
FROM          table_2

```

``` sql

-- Выведите города с максимальным и минимальным весом единичной доставки. Столбцы к выводу — city_name, weight.

(select
c.city_name city_name,
s.weight weight
from
sql.city c
join sql.shipment s on c.city_id = s.city_id
order by 2 desc
limit 1) -- максимум

union all

(select
c.city_name,
s.weight
from
sql.city c
join sql.shipment s on c.city_id = s.city_id
order by 2 asc
limit 1) -- минимум

-- Выведите идентификационные номера клиентов (cust_id), которые совпадают с идентификационными номерами доставок (ship_id). Столбец к выводу — mutual_id. Отсортируйте по возрастанию.

select
c.cust_id mutual_id
from
sql.customer c

intersect

select
s.ship_id
from
sql.shipment s
order by 1

-- Создайте справочник, содержащий уникальные имена клиентов, которые являются производителями (cust_type='manufacturer'), и производителей грузовиков, а также описание объекта — 'КЛИЕНТ' или 'ГРУЗОВИК'. 
-- Столбцы к выводу — object_name, object_description. Отсортируйте по названию в алфавитном порядке.

(select
c.cust_name object_name, 'КЛИЕНТ' object_description
from
sql.customer c
where c.cust_type = 'manufacturer')

union

select
t.make, 'ГРУЗОВИК'
from
sql.truck t

order by 1

```

## Задания на самопроверку

Скачать дата сет можно тут: [скачать](https://lms-cdn.skillfactory.ru/assets/courseware/v1/fb9124cd203bfaf5573ecf478899c6fc/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/SQL_test1.xlsx)

``` sql

-- 1. Вывести список ФИО клиентов, которые покупали продукты компании в этом году
select
c.client_name
from
clients c
join sale_details  sd on c.client_id = sd.sales_details_id
where to_char(sd.date_sale, 'YYYY') = '2024'

-- 2. Вывести список клиентов, у которых день рождения сегодня
select
*
from
clients c
where c.date_of_birth = '23.05.2024'

-- 3. Вывести объём выручки продаж этого года по менеджерам
select
s.manager_id,
s.revenue
from
sales s
where to_char(s.date_sale, 'YYYY') = '2024'

-- 4. Вывести все уникальные сочетания двух категорий продуктов (в рамках одной продажи), которые были куплены сегодня (например: молоко-йогурт, молоко-сосиски, йогурт-сосиски и тд.)
sales
distinct
g_1.category,
g_2.category
from
sales_details sd
join goods g_1 on sd.goods_id = g_1.goods_id
join goods g_2 on sd.goods_id = g_2.goods_id
where sd.date_sale = '23.05.2024'

-- 5. Вывести среднемесячную выручку продаж этого года на каждого менеджера по продажам, в которых была реализация продукта из категории "Колбаса"
select
s.sales_manager,
avg(s.revenue)
from
sales s
join goods g on s.sale_id = g.goods_id
group by s.sales_id, date_trunc('month', s. sale_date)
having to_char(sd.date_sale, 'YYYY') = '2024' and g.category = 'Колбаса'

Примечание: Мной делались примерные скрипты, т.к. дата сет в реальности проверить нельзя. Могут быть ошибки!

```

Дан источник данных — таблица TEST_SQL.

Поля таблицы:

ST_ORD (Varchar) — статус заявки;

TYPE_PRODUCT_NAME (Varchar) — тип заявки;

PRODUCT_INFOSOURCE1 (Varchar) — источник;

CREATE_DATE (Date) — дата создания заявки;

INDEX_LEAD (Number) — индикатор заявки (флаг 0/1 определяет регистрацию лида в системе);

INDEX_ISSUE (Number) — индикатор выдачи (флаг 0/1 определяет наличие выдачи по заявке);

INDEX_SUM (Number) — сумма по продукту.

``` sql

-- 1. Сгруппируйте по месяцам количество заявок и сумму выдач, вычислите долю выдач.

select
to_char(CREATE_DATE, 'YYYY-MM') months,
count(*) total_applications,
sum(INDEX_ISSUE) total_issues,
sum(case when INDEX_SUM = 1 then INDEX_SUM else 0 end) total_issues_sum,
round(sum(INDEX_ISSUES)::float / count(*) * 100, 2) issue_ration_percentage
from
TEST_SQL
group by to_char(CREATE_DATE, 'YYYY-MM')
order by months

-- 2. Определите источник, через который пришло больше всего заявок.

select
PRODUCT_INFOSOURCE1,
count(*) issues_cnt
from
TEST_SQL
group by PRODUCT_INFOSOURCE1
order by issues_cnt desc
limit 1

-- 3. Выделите количество заявок, которые являются ошибочными.

-- Критерии ошибки:

-- Статус заявки не проставляется или имеет значение «тестовая заявка».
-- Заявка была создана в августе 2018 года.
-- Присутствует индикатор выдачи, но при этом лид не был зарегистрирован в системе.

select
count(*)
from
TEST_SQL
when (ST_ORD is null 
or ST_ORD = 'тестовая заявка')
and to_char(CREATE_DATE, 'YYYY-MM') = '2018-август'
and INDEX_ISSUE = 1 
and INDEX_LEAD = 0

-- 4. Каким будет результат выполнения запросов ниже, если в поле CREATE_DATE будет содержаться значение 30.11.2017?

select 
trunc(create_date, 'dd') , create_date 
from 
test_sql -- 30.11.2017 00:00:00

select 
trunc(create_date, 'mm') , create_date 
from 
test_sql -- 01.11.2017 00:00:00

select 
trunc(create_date, 'mm')+1, create_date 
from 
test_sql -- 02.11.2017 00:00:00

```